In [1]:
%%configure -f
{"conf":{
    "spark.pyspark.python":"python3",
    "spark.pyspark.virtualenv.enabled":"true",
    "spark.pyspark.virtualenv.type":"native",
    "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    
    "spark.executor.heartbeaInterval":"10000s",
    "spark.network.timeout":"24h",
    
    "spark.driver.memory":"1G",
    "spark.executor.memory":"1G",
    "spark.executor.cores":"2",

    "spark.app.name":"msds694"
}
}



In [ ]:
# import pyspark
# from pyspark import SparkContext

import plotly.io as pio
pio.renderers
pio.renderers.default = "browser"
from datetime import datetime

import plotly
import plotly.graph_objects as go

In [2]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1638934593238_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=msds694>

In [3]:
sc.install_pypi_package("plotly")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version  
-------------------------- ---------
aws-cfn-bootstrap          2.0      
beautifulsoup4             4.9.3    
boto                       2.49.0   
click                      7.1.2    
docutils                   0.14     
jmespath                   0.10.0   
joblib                     1.0.1    
lockfile                   0.11.0   
lxml                       4.6.3    
mysqlclient                1.4.2    
nltk                       3.5      
nose                       1.3.4    
numpy                      1.16.5   
pip                        9.0.1    
plotly                     5.4.0    
py-dateutil                2.2      
pystache                   0.5.4    
python-daemon              2.2.3    
python37-sagemaker-pyspark 1.4.1    
pytz                       2021.1   
PyYAML                     5.4.1    
regex                      2021.3.17
setuptools                 28.8.0   
simplejson                 3.2.0    
six                        1.13.0   
t

In [7]:
data_path_london = "s3://msds0694/london.csv"
data_path_station = "s3://msds0694/london_stations.csv"
data_london = sc.textFile(data_path_london)
data_london_station = sc.textFile(data_path_station)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
data_london_station.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'station_id,station_name,longitude,latitude'

In [ ]:
rdd = sc.textFile('london.csv')
header = rdd.first()
print(header)
header_rdd = sc.parallelize([header])
data_rdd = rdd.subtract(header_rdd)
print()
map_rdd = sc.textFile('london_stations.csv')
header_map = map_rdd.first()
print(header_map)
maphead = sc.parallelize([header_map])
data_map = map_rdd.subtract(maphead)

In [10]:
def parse_csv(data):
    pre_lst=data.split('"')
    if len(pre_lst)==1:
        return data.split(',')
    return_data = []
    for i, l in enumerate(pre_lst):
        if i == 0:
            return_data += l.split(',')[:-1]
        elif i == len(pre_lst)-1:
            return_data += l.split(',')[1:]
        elif i%2 == 0:
            return_data += l.split(',')[1:-1]
        else:
            return_data += [l]
    return return_data
            

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## get tidy rdd
data_tidy = data_rdd.map(lambda x:parse_csv(x))
data_tidy.cache()

map_tidy = data_map.map(lambda x:parse_csv(x))
map_tidy.cache()

In [ ]:
#get paired rdd
map_pair = map_tidy.map(lambda x:(x[0],x[1:]))
startstation_pair = data_tidy.map(lambda x:(x[-2],x[-1]))
start_loc = startstation_pair.join(map_pair)

In [6]:
%%local
import plotly
import plotly.graph_objects as go